# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [1]:
#include <stdio.h>

int main(){
    float s = 200;
    int numParcelas = 5;
    float juros = 0.01;
    
    
    printf("Parcelas do financiamento: ");
    for(int i = 0; i < numParcelas; i++){
        printf("%.2f; ", s);
        s += s * juros;
    }
    
    
    return 0;
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [12]:
#include <stdio.h>

float proximaParcela(float parcelaAtual, float juros){
    float novaParcela = parcelaAtual + parcelaAtual * juros;
    return novaParcela;
}

int main(){
    float s = 200;
    int numParcelas = 5;
    float juros = 0.01;
    
    printf("Parcelas do financiamento: ");
    for(int i = 0; i < numParcelas; i++){
        printf("%.2f; ", s);
        s = proximaParcela(s, juros);
    }
    
    
    return 0;
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [10]:
#include <stdio.h>
#include <stdlib.h>

void calcularEmprestimo(float parcelaInicial, float juros, int numParcelas, float* valorParcelas){
    
    valorParcelas[0] = parcelaInicial;
    
    for(int i = 1; i < numParcelas; i++){
        valorParcelas[i] = valorParcelas[i-1] + valorParcelas[i-1] * juros;
    }
}

void imprimirProximaParcela(float *parcelas, int ind){
    printf("%.2f; ", parcelas[ind]);
}

int main(){
    float s = 200;
    int numParcelas = 5;
    float juros = 0.01;
    float* valorParcelas;
    
    valorParcelas = malloc(sizeof(float) * numParcelas);
    
    calcularEmprestimo(s, juros, numParcelas, valorParcelas);
    
    printf("Parcelas do financiamento: ");
    for(int i = 0; i < numParcelas; i++){
        imprimirProximaParcela(valorParcelas, i);
    }
    
    free(valorParcelas);
    
    return 0;
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [17]:
#include <stdio.h>

typedef struct{
    float s;
    int numParcelas;
    float juros;
    float parcelaAtual;
}Emprestimo;


Emprestimo proximaParcela(Emprestimo emp){
    emp.parcelaAtual = emp.parcelaAtual + emp.parcelaAtual * emp.juros;
    return emp;
}

int main(){
    Emprestimo emp;

    emp.s = 200;
    emp.numParcelas = 5;
    emp.juros = 0.01;
    
    emp.parcelaAtual = emp.s;
    printf("Parcelas do financiamento: ");
    for(int i = 0; i < emp.numParcelas; i++){
        printf("%.2f; ", emp.parcelaAtual);
        emp = proximaParcela(emp);
    }
    
    
    return 0;
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [1]:
#include <stdio.h>
#include <stdlib.h>

typedef struct{
    float s;
    int numParcelas;
    float juros;
    float parcelaAtual;
    int numParcelaAtual;
}Emprestimo;

Emprestimo iniciarEmprestimo(float s, int numParcelas, float juros){
    Emprestimo novoEmp;
    
    novoEmp.s = s;
    novoEmp.numParcelas = numParcelas;
    novoEmp.juros = juros;
    novoEmp.parcelaAtual = s;
    novoEmp.numParcelaAtual = 1;
    
    return novoEmp;
}

Emprestimo pegar_mais_longo(Emprestimo* emps, int numEmps){
    
    Emprestimo mais_longo = emps[0];
    
    for(int i = 1; i < numEmps; i++){
        Emprestimo candidato = emps[i];
        
        if(mais_longo.numParcelas < candidato.numParcelas){
            mais_longo = candidato;
        }
    }
    
    return mais_longo;
}

void proximaParcela(Emprestimo* emps, int ind){
    if(emps[ind].numParcelaAtual <= emps[ind].numParcelas){
        printf("Emprestimo %d: parcela %d eh %.2f\n", ind+1, emps[ind].numParcelaAtual, emps[ind].parcelaAtual);
        emps[ind].numParcelaAtual++;
        emps[ind].parcelaAtual += emps[ind].parcelaAtual * emps[ind].juros;
    }
}

int main(){
    Emprestimo* emprestimos;
    
    // Passar o valor dos emprestimos e aumentar o numEmprestimos
    int numEmprestimos = 2;
    emprestimos = malloc(sizeof(Emprestimo) * numEmprestimos);
    
    emprestimos[0] = iniciarEmprestimo(200, 5, 0.01);
    emprestimos[1] = iniciarEmprestimo(500, 7, 0.02);
    //emprestimos[2] = iniciarEmprestimo(700, 9, 0.03);
    
    Emprestimo emprestimo_mais_longo = pegar_mais_longo(emprestimos, numEmprestimos);
    for(int i = 0; i < emprestimo_mais_longo.numParcelas; i++){
        for(int indEmp = 0; indEmp < numEmprestimos; indEmp++){
            proximaParcela(emprestimos, indEmp);
        }
    }
    
    free(emprestimos);
    
    return 0;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 3: parcela 1 eh 700.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 3: parcela 2 eh 721.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 3: parcela 3 eh 742.63
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 3: parcela 4 eh 764.91
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 3: parcela 5 eh 787.86
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 3: parcela 6 eh 811.49
Emprestimo 2: parcela 7 eh 563.08
Emprestimo 3: parcela 7 eh 835.84
Emprestimo 3: parcela 8 eh 860.91
Emprestimo 3: parcela 9 eh 886.74
